In [1]:
import logging
from arango import ArangoClient  # Python-Arango driver
from torch_geometric.datasets import FakeHeteroDataset # Sample graph form PyG
import adbpyg_adapter
from adbpyg_adapter import ADBPyG_Adapter

In [2]:
import pandas as pd
from adb_cloud_connector import utils
from arango import ArangoClient
import time
import yaml
from yaml.loader import SafeLoader
import os
import traceback


In [3]:
def create_movie_db():
    # The names of the graph are string literals now, can be made configurable in a subsequent iteration
    GRAPH_NAME = "IMDB_MOVIE_GRAPH"
    con = utils.get_temp_credentials()
    print("CONNECTION: " + str(con))
    db = ArangoClient(hosts=con["url"]).db(con["dbName"], con["username"], con["password"], verify=True)

    if not db.has_graph(GRAPH_NAME):
        db.create_graph(GRAPH_NAME)

    if not db.has_graph(GRAPH_NAME):
        db.create_graph(GRAPH_NAME)
    MOVIE_COLL = "MOVIE"
    REPLICATION_FACTOR = 3
    if not db.has_collection(MOVIE_COLL):
        db.create_collection(MOVIE_COLL, REPLICATION_FACTOR)

    USER_COLL= "USER"
    if not db.has_collection(USER_COLL):
        db.create_collection(USER_COLL, REPLICATION_FACTOR)

    imdb_graph = db.graph(GRAPH_NAME)
    EDGE_COLL = "RATINGS"
    if not db.has_collection(EDGE_COLL):
        db.create_collection(EDGE_COLL, REPLICATION_FACTOR, edge=True)

    if not imdb_graph.has_edge_definition(EDGE_COLL):
        rating_value = imdb_graph.create_edge_definition(
            edge_collection=EDGE_COLL,
            from_vertex_collections=[USER_COLL],
            to_vertex_collections=[MOVIE_COLL]
        )

    print("Done provisioning database!")

    return db

In [4]:
def load_users(db):
    print("Loading Users...")
    st = time.time()
    USER_COLL = "USER"
    uc = db[USER_COLL]
    fp = "data/u.user"
    df = pd.read_csv(fp, sep="|", header = None)
    df.columns = ["userID", "age", "gender", "occupation", "zip"]
    BATCH_SIZE = 500
    batch_docs = []
    for index, row in df.iterrows():
        doc = {'_id': "USER/" + str(row["userID"]),\
               'age': int(row["age"]), 'gender': str(row["gender"]),\
               'occupation': str(row["occupation"]), 'zip': str(row["zip"])}
        batch_docs.append(doc)
    result = uc.import_bulk(batch_docs, on_duplicate= "update", batch_size = BATCH_SIZE)
    et = time.time()
    lt = (et - st)/60
    print("Done with Users! Load took {:.2f} minutes.".format(lt))
    return result

In [5]:

def load_movies(db):
    print("Loading Movies...")
    st = time.time()
    MOVIE_COLL = "MOVIE"
    mc = db[MOVIE_COLL]
    fp = "data/u.item.fe"
    df = pd.read_csv(fp)

    movie_attr_cols = ["itemID", "title", "release_date", "IMDB_URL", "G_UNKNOWN", "G_ACTION",\
                  "G_ADVENTURE", "G_ANIMATION", "G_CHILDREN", "G_COMEDY", "G_CRIME", "G_DOCUMENTARY", "G_DRAMA",\
                  "G_FANTASY", "G_FILM_NOIR", "G_HORROR", "G_MUISICAL", "G_MYSTERY", "G_ROMANCE", "G_SCIFI", "G_THRILLER",\
                  "G_WAR", "G_WESTERN", "rating", "poor_rating"]
    df.columns = movie_attr_cols
    BATCH_SIZE = 500
    batch_docs = []
    genre_cols = ["G_UNKNOWN", "G_ACTION",\
                  "G_ADVENTURE", "G_ANIMATION", "G_CHILDREN", "G_COMEDY", "G_CRIME", "G_DOCUMENTARY", "G_DRAMA",\
                  "G_FANTASY", "G_FILM_NOIR", "G_HORROR", "G_MUISICAL", "G_MYSTERY", "G_ROMANCE", "G_SCIFI", "G_THRILLER",\
                  "G_WAR", "G_WESTERN"]
    for index, row in df.iterrows():
        doc = {}
        genre_attr = []
        for col in movie_attr_cols:
            if col == "itemID":
                doc["_id"] = MOVIE_COLL + "/" + str(row[col])
            elif col in genre_cols:
                genre_attr.append(int(row[col]))
            else:
                doc[col] = str(row[col])
        doc['genre'] = genre_attr
        batch_docs.append(doc)
    result = mc.import_bulk(batch_docs, on_duplicate= "update", batch_size = BATCH_SIZE)
    et = time.time()
    lt = (et - st)/60

    print("Done with Movies! Load took {:.2f} minutes".format(lt))
    return result

In [6]:
def load_ratings(db):
    print("Loading Ratings...")
    EDGE_COLL = "RATINGS"
    USER_COLL = "USER"
    MOVIE_COLL = "MOVIE"
    rc = db[EDGE_COLL]
    fp = "data/u.data"
    df = pd.read_csv(fp, sep="\t")
    df.columns = ["userID", "itemID", "rating", "timestamp"]
    BATCH_SIZE = 500
    batch_docs = []
    for index, row in df.iterrows():
        doc = {'_from': USER_COLL+ "/"+ str(row["userID"]), "_to": MOVIE_COLL+ "/"+ str(row["itemID"]), "rating": str(row["rating"])}
        batch_docs.append(doc)
    result = rc.import_bulk(batch_docs, on_duplicate= "update", batch_size = BATCH_SIZE)
    print("Done with Ratings!")
    return result


In [7]:

def load_data(db):
    st = time.time()
    result = load_users(db)
    result = load_movies(db)
    result = load_ratings(db)
    et = time.time()
    lt = (et-st)/60
    print("Execution time : {:.2f} minutes".format(lt))
    return

In [8]:
db = create_movie_db()

Log: requesting new credentials...
Succcess: new credentials acquired
CONNECTION: {'dbName': 'TUTnr4e9pqm0abvnnwk1o319g', 'username': 'TUTxih82oxf98g4qmdque2ic', 'password': 'TUT5eelihslerxqmmb0v2d4zc', 'hostname': 'tutorials.arangodb.cloud', 'port': 8529, 'url': 'https://tutorials.arangodb.cloud:8529'}
Done provisioning database!


In [9]:
load_data(db)

Loading Users...
Done with Users! Load took 0.02 minutes.
Loading Movies...
Done with Movies! Load took 0.03 minutes
Loading Ratings...
Done with Ratings!
Execution time : 1.09 minutes


In [10]:
adbpyg_adapter = ADBPyG_Adapter(db, logging_lvl=logging.DEBUG)

[2022/07/20 05:20:29 +0530] [12512] [INFO] - adbpyg_adapter: Instantiated ADBPyG_Adapter with database 'TUTnr4e9pqm0abvnnwk1o319g'


In [11]:
pyg_fraud_graph_2 = adbpyg_adapter.arangodb_collections_to_pyg(
    "IMDB_MOVIE_GRAPH",
    {"MOVIE", "USER"},  # Vertex collections
    {"RATINGS"},  # Edge collections
)

[2022/07/20 05:20:30 +0530] [12512] [DEBUG] - adbpyg_adapter: --arangodb_to_pyg('IMDB_MOVIE_GRAPH')--
[2022/07/20 05:20:30 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 'USER' vertices
[2022/07/20 05:20:32 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 'MOVIE' vertices
[2022/07/20 05:20:32 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 'RATINGS' edges
[2022/07/20 05:20:57 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 99999 '('USER', 'RATINGS', 'MOVIE')' edges
[2022/07/20 05:20:57 +0530] [12512] [INFO] - adbpyg_adapter: Created PyG 'IMDB_MOVIE_GRAPH' Graph


In [13]:
from adbpyg_adapter.utils import IdentityEncoder, EnumEncoder

In [27]:
import torch
from sentence_transformers import SentenceTransformer
class SequenceEncoder(object):
    def __init__(self,model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [29]:
import torch
def fancy_callback_fn(df):
    user_occupation = df["occupation"]
    uot = SequenceEncoder()(user_occupation)
    return uot
    

In [37]:
metagraph = { 
    "vertexCollections": {
        "USER": {'x': {"age":IdentityEncoder()}},#{"age": IdentityEncoder()}},
        "MOVIE": {'x': {"title": SequenceEncoder(), "genre": None}, 'y': {'poor_rating':EnumEncoder() }}
    },
    "edgeCollections": {
        "RATINGS": {},
    },
}

In [38]:
pyg_fraud_graph_3 = adbpyg_adapter.arangodb_to_pyg("IMDB_MOVIE_GRAPH", metagraph)

[2022/07/20 05:56:09 +0530] [12512] [DEBUG] - adbpyg_adapter: --arangodb_to_pyg('IMDB_MOVIE_GRAPH')--
[2022/07/20 05:56:09 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 'USER' vertices
[2022/07/20 05:56:10 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 'MOVIE' vertices


Batches:   0%|          | 0/53 [00:00<?, ?it/s]

[2022/07/20 05:56:14 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 'RATINGS' edges
[2022/07/20 05:56:39 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 99999 '('USER', 'RATINGS', 'MOVIE')' edges
[2022/07/20 05:56:39 +0530] [12512] [INFO] - adbpyg_adapter: Created PyG 'IMDB_MOVIE_GRAPH' Graph


In [25]:
pyg_fraud_graph_3

HeteroData(
  USER={ x=[943, 1] },
  MOVIE={
    x=[1682, 403],
    y=[1682, 1]
  },
  (USER, RATINGS, MOVIE)={ edge_index=[2, 99999] }
)

In [40]:
adbpyg_adapter.pyg_to_arangodb("IMDB_MOVIE_GRAPH_2", pyg_fraud_graph_3)

[2022/07/20 05:59:12 +0530] [12512] [DEBUG] - adbpyg_adapter: --pyg_to_arangodb('IMDB_MOVIE_GRAPH_2')--
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: Preparing 1682 'MOVIE' nodes
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N0: 0
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N1: 1
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N2: 2
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N3: 3
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N4: 4
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N5: 5
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N6: 6
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N7: 7
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N8: 8
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N9: 9
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N10: 10
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_ad

[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N115: 115
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N116: 116
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N117: 117
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N118: 118
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N119: 119
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N120: 120
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N121: 121
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N122: 122
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N123: 123
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N124: 124
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N125: 125
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N126: 126
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N127: 127
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N1

[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N229: 229
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N230: 230
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N231: 231
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N232: 232
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N233: 233
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N234: 234
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N235: 235
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N236: 236
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N237: 237
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N238: 238
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N239: 239
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N240: 240
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N241: 241
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N2

[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N343: 343
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N344: 344
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N345: 345
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N346: 346
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N347: 347
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N348: 348
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N349: 349
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N350: 350
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N351: 351
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N352: 352
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N353: 353
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N354: 354
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N355: 355
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N3

[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N457: 457
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N458: 458
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N459: 459
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N460: 460
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N461: 461
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N462: 462
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N463: 463
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N464: 464
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N465: 465
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N466: 466
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N467: 467
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N468: 468
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N469: 469
[2022/07/20 05:59:13 +0530] [12512] [DEBUG] - adbpyg_adapter: N4

...

[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1249: 1249
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1250: 1250
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1251: 1251
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1252: 1252
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1253: 1253
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1254: 1254
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1255: 1255
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1256: 1256
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1257: 1257
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1258: 1258
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1259: 1259
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1260: 1260
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1261: 1261
[2022/07/20 05:59:14 +0530] [12512] [D

[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1360: 1360
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1361: 1361
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1362: 1362
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1363: 1363
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1364: 1364
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1365: 1365
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1366: 1366
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1367: 1367
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1368: 1368
[2022/07/20 05:59:14 +0530] [12512] [DEBUG] - adbpyg_adapter: N1369: 1369
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1370: 1370
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1371: 1371
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1372: 1372
[2022/07/20 05:59:15 +0530] [12512] [D

[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1471: 1471
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1472: 1472
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1473: 1473
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1474: 1474
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1475: 1475
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1476: 1476
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1477: 1477
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1478: 1478
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1479: 1479
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1480: 1480
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1481: 1481
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1482: 1482
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1483: 1483
[2022/07/20 05:59:15 +0530] [12512] [D

[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1582: 1582
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1583: 1583
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1584: 1584
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1585: 1585
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1586: 1586
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1587: 1587
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1588: 1588
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1589: 1589
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1590: 1590
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1591: 1591
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1592: 1592
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1593: 1593
[2022/07/20 05:59:15 +0530] [12512] [DEBUG] - adbpyg_adapter: N1594: 1594
[2022/07/20 05:59:15 +0530] [12512] [D

DocumentInsertError: [HTTP 409][ERR 1210] unique constraint violated